## Imports

In [54]:
import torch.nn as nn
from data.Data import Data
import torch
import numpy as np
import torch.optim as optim

## SEED

In [55]:
import random

SEED = None
if not SEED:
  SEED = random.randint(0, 10000)
print(f"SEED: {SEED}")

np.random.seed(seed=SEED)
random.seed(SEED)

SEED: 7532


## Datasets

In [56]:
dataset = "breast-cancer"
COLUMNS_FOR_BREAST_CANCER = ['node-caps', 'inv-nodes', 'tumor-size', 'deg-malig', 'irradiat', 'class']

In [57]:
distinct_columns = None
if dataset == "breast-cancer":
  distinct_columns = COLUMNS_FOR_BREAST_CANCER

data = Data(dataset, distinct_columns=distinct_columns)

## Train, evaluate, test datasets

In [58]:
train, test = data.get_train_and_valid_set(0.7)
train_x = train.drop('class', axis=1)
train_y = train['class']

test_x = test.drop('class', axis=1)
test_y = test['class']

In [59]:
NUMBER_INPUTS = train_x.shape[1]

In [60]:
print(f"train_0: {len(train[train['class'] == 0])}, train_1: {len(train[train['class'] == 1])}")
print(f"test_0: {len(test[test['class'] == 0])}, test_1: {len(test[test['class'] == 1])}")

train_0: 141, train_1: 59
test_0: 60, test_1: 26


## Data loaders

In [61]:
BATCH_SIZE = 64

def one_hot_class(dataset):
  dataset = np.array(dataset)
  return torch.tensor([[int(i == x) for i in range(2)] for x in dataset])

train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(np.array(train_x)), one_hot_class(train_y))
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(np.array(test_x)), torch.from_numpy(np.array(test_y)))

## Sampling data because of imbalance

In [62]:
from torch.utils.data import WeightedRandomSampler

classes = {0: 0.2, 1: 0.6}
y_classified = [int(np.argmax(y)) for _, y in train_dataset]
example_weights = [classes[e] for e in y_classified]

sampler = WeightedRandomSampler(example_weights, len(train_dataset))

In [63]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler)

## Hyperparameter tuning

In [64]:
import optuna

In [65]:
def define_model(trial):
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []
    
    in_features = NUMBER_INPUTS
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 50, 1000)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.BatchNorm1d(out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_uniform("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, 2))

    return nn.Sequential(*layers)
    
    

## ==== Evaluate model function ====

In [66]:
models_dir = './model_saves/nn_models_saves/'

def get_accuracy(model, test_dataset, save_if_good=False):
  true_preds, num_preds = 0., 0.

  model.eval()
  with torch.no_grad(): # Deactivate gradients for the following code
      for inputs, labels in test_dataset:
          inputs = torch.transpose(torch.tensor(inputs).unsqueeze(1), 0, 1)
          preds = model(inputs.float())
          preds = torch.sigmoid(preds)
          preds = preds.numpy()
          preds = preds.squeeze()

          preds = [int (x == max(preds)) for x in preds]
          true_preds += (np.argmax(preds)) == np.argmax(labels.numpy()).sum()
          num_preds += 1

  acc = true_preds / num_preds
  if save_if_good and acc > 0.8:
     torch.save(model, f"{models_dir}model_{dataset}_{acc:4.2f}.pt")
  return acc

In [67]:
def objective(trial):
    # Generate the model.
    model = define_model(trial)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    MSELoss = nn.MSELoss()
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    NUMBER_EPOCHS = trial.suggest_int("number_epochs", 5, 150)
    model.train()
    for epoch in range(NUMBER_EPOCHS):  # loop over the dataset multiple times
        for inputs, labels in train_data_loader:
            # get the inputs; data is a list of [inputs, labels]

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            preds = model(inputs.float())
            preds = torch.sigmoid(preds)
            loss = MSELoss(preds, labels.float())
            
            loss.backward()
            optimizer.step()
    
    return get_accuracy(model=model, test_dataset=test_dataset, save_if_good=True)
    

In [68]:
study = optuna.create_study(direction="maximize",
                            sampler=optuna.samplers.TPESampler(seed=SEED),
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=10))
study.optimize(objective, n_trials=100, show_progress_bar=True)

pruned_trials = [t for t in study.trials if t.state == optuna.TrialPruned]

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))

[I 2023-06-13 10:23:25,326] A new study created in memory with name: no-name-4fea6719-16ee-40f4-b85e-06a808a8e6a1


  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\User\AppData\Local\Temp/ipykernel_23060/3653047830.py:11: FutureWarning:

suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.

C:\Users\User\AppData\Local\Temp/ipykernel_23060/2870069813.py:9: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



[I 2023-06-13 10:23:26,021] Trial 0 finished with value: 0.47674418604651164 and parameters: {'n_layers': 3, 'n_units_l0': 690, 'dropout_l0': 0.3422635918777778, 'n_units_l1': 113, 'dropout_l1': 0.44374656239806165, 'n_units_l2': 194, 'dropout_l2': 0.31985909744183727, 'optimizer': 'RMSprop', 'lr': 5.169961738133986e-05, 'number_epochs': 27}. Best is trial 0 with value: 0.47674418604651164.
[I 2023-06-13 10:23:27,013] Trial 1 finished with value: 0.5813953488372093 and parameters: {'n_layers': 2, 'n_units_l0': 634, 'dropout_l0': 0.3116059777358236, 'n_units_l1': 576, 'dropout_l1': 0.30894082151990937, 'optimizer': 'RMSprop', 'lr': 0.0013944207581020186, 'number_epochs': 39}. Best is trial 1 with value: 0.5813953488372093.
[I 2023-06-13 10:23:30,273] Trial 2 finished with value: 0.38372093023255816 and parameters: {'n_layers': 3, 'n_units_l0': 922, 'dropout_l0': 0.4683463568367692, 'n_units_l1': 312, 'dropout_l1': 0.36740101462538916, 'n_units_l2': 795, 'dropout_l2': 0.33022110917258307

In [69]:
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value:  0.8488372093023255
  Params: 
    n_layers: 2
    n_units_l0: 435
    dropout_l0: 0.37266774325989
    n_units_l1: 519
    dropout_l1: 0.23654368123432648
    optimizer: RMSprop
    lr: 0.0028440828964250227
    number_epochs: 49


In [70]:
optuna.visualization.plot_optimization_history(study)

In [71]:
optuna.visualization.plot_param_importances(study)

## Classifier according to hyperparameter tuning

In [72]:
model = torch.load(f"{models_dir}model_{dataset}.pt")

In [73]:
# class SimpleClassifier(nn.Module):

#     def __init__(self, num_inputs, num_outputs):
#         super().__init__()
#         # Initialize the modules we need to build the network
#         self.lin1 = nn.Linear(num_inputs, trial.params["n_units_l0"])
#         self.bn1 = nn.BatchNorm1d(trial.params["n_units_l0"])
#         self.act1 = nn.ReLU()
#         self.d1 = nn.Dropout(trial.params["dropout_l0"])
        
#         self.lin2 = nn.Linear(trial.params["n_units_l0"], trial.params["n_units_l1"])
#         self.bn2 = nn.BatchNorm1d(trial.params["n_units_l1"])
#         self.act2 = nn.ReLU()
#         self.d2 = nn.Dropout(trial.params["dropout_l1"])
        
#         self.lin3 = nn.Linear(trial.params["n_units_l1"], num_outputs)

#     def forward(self, x):
#         # Perform the calculation of the model to determine the prediction
#         x = self.lin1(x)
#         x = self.bn1(x)
#         x = self.act1(x)
#         x = self.d1(x)
        
#         x = self.lin2(x)
#         x = self.bn2(x)
#         x = self.act2(x)
#         x = self.d2(x)

#         x = self.lin3(x)
#         return x

## Train model

In [74]:
# NUMBER_EPOCHS = trial.params["number_epochs"]

# MSELoss = nn.MSELoss()
# optimizer = optim.RMSprop(model.parameters(), lr=trial.params["lr"])

# model.train()

# for epoch in range(NUMBER_EPOCHS):  # loop over the dataset multiple times
    
#     sum_loss = 0

#     for inputs, labels in train_data_loader:
#         # get the inputs; data is a list of [inputs, labels]

#         # zero the parameter gradients
#         optimizer.zero_grad()

#         # forward + backward + optimize
#         preds = model(inputs.float())
#         preds = torch.sigmoid(preds)
#         loss = MSELoss(preds, labels.float())
        
#         loss.backward()
#         optimizer.step()

#         sum_loss += loss
    
#     print(epoch, sum_loss / X_train.shape[0])

## Train set evaluation

In [77]:
model.eval()

acc = get_accuracy(model, test_dataset=train_dataset)
print(f"Accuracy of the model: {100.0*acc:4.2f}%")

Accuracy of the model: 82.50%


C:\Users\User\AppData\Local\Temp/ipykernel_23060/2870069813.py:9: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).



## Test set evaluation

In [78]:
model.eval()

acc = get_accuracy(model, test_dataset=test_dataset)
print(f"Accuracy of the model: {100.0*acc:4.2f}%")

Accuracy of the model: 63.95%


C:\Users\User\AppData\Local\Temp/ipykernel_23060/2870069813.py:9: UserWarning:

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).

